# 3S_{tr2}(2) в зависимости от плотности теплоносителя, температуры теплоносителя, температуры топлива и концентрации борной кислоты

## 1. Загрузка данных

In [1]:
import pandas as pd

Загрузка данных

In [2]:
df = pd.read_excel('Input/Data.xlsx')
df.head(3)

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),3S_{tr1}(1),3S_{tr2}(2),\Sigma_{a1}(3),\Sigma_{a2}(4),nuS_{f1}(5),nuS_{f2}(6),\Sigma_{f1}(7),\Sigma_{f2}(8),\Sigma_{1->2}(9),k_{inf}(10),\sigma_{a2}^{Xe},\sigma_{a2}^{Sm}
0,0.7235,575.0,1027.0,600.0,0.661853,2.568298,0.009376,0.085584,0.006677,0.133250,0.002620,0.054750,0.017392,1.261041,1072970.0,35421.398
1,0.7750,448.0,1043.0,1500.0,0.679602,2.874687,0.009581,0.098339,0.006723,0.139256,0.002636,0.057218,0.018726,1.174265,1141000.0,36147.801
2,0.9125,370.5,1418.0,750.0,0.727336,3.668742,0.009908,0.102815,0.006844,0.153823,0.002681,0.063203,0.022463,1.249619,1308480.0,39214.000


In [3]:
names = ['\\rho(g/cm^3)',
 'T_c(K)',
 'T_f(K)',
 'c_b(ppm)',
 '3S_{tr2}(2)']

In [4]:
df = df[names]
df.head(3)

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),3S_{tr2}(2)
0,0.7235,575.0,1027.0,600.0,2.568298
1,0.7750,448.0,1043.0,1500.0,2.874687
2,0.9125,370.5,1418.0,750.0,3.668742


### 2 Нормализация признаков

In [5]:
features = ['\\rho(g/cm^3)',
 'T_c(K)',
 'T_f(K)',
 'c_b(ppm)']

In [6]:
df_features_norm = df.copy()

In [7]:
for i in range(len(features)):
    df_features_norm[features[i]] = (df[features[i]]-df[features[i]].mean())/df[features[i]].std()

In [8]:
df_features_norm

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),3S_{tr2}(2)
0,-0.316426,1.416398,-0.039094,-1.035019,2.568298
1,0.007922,-0.005506,-0.001946,0.006984,2.874687
2,0.873902,-0.873204,0.868693,-0.861352,3.668742
3,-0.858058,0.862191,-0.872586,0.875321,2.154513
4,-0.425067,-0.439355,0.433373,-1.295520,2.588662
...,...,...,...,...,...
252,-0.844527,-0.886762,-1.539169,-0.603564,2.259366
253,0.887433,0.848634,0.202111,1.133108,3.475698
254,0.021453,-0.019064,1.072750,-1.471900,2.925234
255,-1.710507,1.716332,-0.668529,0.264771,1.505131


In [9]:
df_features_norm.sample(frac=1) 

,\rho(g/cm^3),T_c(K),T_f(K),c_b(ppm),3S_{tr2}(2)
137,0.698000,0.062283,-1.403131,0.888889,3.388062
35,-0.695686,0.048725,1.249596,-0.589997,2.333445
44,-0.479192,-0.168199,-0.274021,-1.675417,2.538348
163,-0.655093,1.092673,-1.457546,0.508992,2.302630
158,0.156763,-0.480029,-1.294301,-0.739242,3.044863
...,...,...,...,...,...
108,-0.506253,-1.496862,-1.335113,1.010999,2.522914
130,0.914495,-1.456189,-1.620791,0.671805,3.686024
125,0.900964,1.431618,-0.790963,-1.268384,3.509166
139,-0.167980,0.929980,1.208786,0.020553,2.687182


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_features_norm[['\\rho(g/cm^3)', 'T_c(K)', 'T_f(K)', 'c_b(ppm)']],
                                                    df_features_norm[['3S_{tr2}(2)']],
                                                    test_size=0.33,
                                                    random_state=42)

# Нейросетевые модели

In [11]:
%%time

# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf

# Function to create model, required for KerasRegressor
def create_model(activation='relu', learn_rate=0.01, momentum=0):
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=4, activation=activation))
    model.add(Dense(1, kernel_initializer='normal'))
    
    optimizer = tf.keras.optimizers.RMSprop(learning_rate = learn_rate, rho = 0.99, momentum = momentum, epsilon=1e-09)
    
    # Compile model
    model.compile(loss='mse', optimizer=optimizer)
    return model

X = X_train
Y = y_train

# create model
model = KerasRegressor(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [10, 15, 20]
epochs = [1000, 1200, 1300]
activation = ['tanh', 'sigmoid', 'linear']
learn_rate = [0.00015, 0.0002, 0.00025]
momentum = [0, 0.00001]

param_grid = dict(batch_size=batch_size,
                  epochs=epochs,
                  activation=activation,
                  learn_rate=learn_rate,
                  momentum=momentum
                 )
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=5)
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.000624 using {'activation': 'linear', 'batch_size': 15, 'epochs': 1000, 'learn_rate': 0.00015, 'momentum': 0}
-0.016388 (0.008940) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 1000, 'learn_rate': 0.00015, 'momentum': 0}
-0.015872 (0.003971) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 1000, 'learn_rate': 0.00015, 'momentum': 1e-05}
-0.008320 (0.001612) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 1000, 'learn_rate': 0.0002, 'momentum': 0}
-0.007634 (0.004515) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 1000, 'learn_rate': 0.0002, 'momentum': 1e-05}
-0.004672 (0.001575) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 1000, 'learn_rate': 0.00025, 'momentum': 0}
-0.004577 (0.003248) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 1000, 'learn_rate': 0.00025, 'momentum': 1e-05}
-0.011657 (0.003153) with: {'activation': 'tanh', 'batch_size': 10, 'epochs': 1200, 'learn_rate': 0.00015, 'momentum': 0}
-0.010943 (0.0043

In [12]:
activation': 'linear', 'batch_size': 15, 'epochs': 1000, 'learn_rate': 0.00015, 'momentum': 0}

NameError: name 'stop' is not defined

## Модель 1

In [25]:
def create_model():

    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=4, activation='linear'))
    model.add(Dense(1, kernel_initializer='normal'))

    optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.00015, rho = 0.99, momentum = 0, epsilon=1e-09)
    
    # Compile model
    model.compile(loss='mse', optimizer=optimizer)
    return model

In [26]:
model = create_model()
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Обучение модели

EPOCHS = 1000
batch_size = 15

history = model.fit(
  X, Y,
  epochs=EPOCHS, batch_size = batch_size,verbose=0)

In [28]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail(10)

,loss,epoch
990,0.000602,990
991,0.000603,991
992,0.000602,992
993,0.000603,993
994,0.000597,994
995,0.000601,995
996,0.000607,996
997,0.000605,997
998,0.000601,998
999,0.000612,999


In [29]:
import statsmodels as statsmodels
import statsmodels.api as sm

RMSPE = statsmodels.tools.eval_measures.rmspe(y_test.values, model.predict(X_test))
RMSPE

array([0.12749358])

In [30]:
import statsmodels as statsmodels
import statsmodels.api as sm

RMSPE = statsmodels.tools.eval_measures.rmspe(y_train.values, model.predict(X_train))
RMSPE

array([0.10080699])